In [ ]:
import torch
import torch.nn as nn

In [ ]:
#LeNet Architecture
# (1,32,32) -> (5x5),s=1,p=0,c=6 -> avg pool (2x2),s=2 -> (5x5),s=1,p=0,c=16 -> avg pool (2x2),s=2 ->
# Conv 5x5 to 120 channels-> Linear 120 -> Linear 84 -> Linear 10
class LeNet(nn.Module):
    def __init__(self, in_channels=1):
        super(LeNet, self).__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=(5,5), stride=(1,1), padding=(0,0))
        self.pool = nn.AvgPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=(1,1), padding=(0,0))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5,5), stride=(1,1), padding=(0,0))
        self.fc1 = nn.Linear(120, 84)
        self.out = nn.Linear(84, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)

        x = self.relu(self.conv2(x))
        x = self.pool(x)

        x = self.relu(self.conv3(x)) #num_examples x 120 x 1 x 1 -> num_examples x 120
        x = x.reshape(x.shape[0], -1)

        x = self.relu(self.fc1(x))
        x = self.out(x)
        return x

In [ ]:
x = torch.rand(64, 1, 32, 32)
model = LeNet()
print(model(x).shape)

torch.Size([64, 10])


In [ ]:
#VGG16 Architecture
vgg_16 = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
#Then flatten and 4096 -> 4096 -> 1000 Linear layers
class VGG16(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(VGG16, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.conv_layers = self.create_conv_layers(vgg_16)
        self.fc_layers = self.create_fc_layers()

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc_layers(x)
        return x
    
    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            out_channels = x
            if type(x) == int:
                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                                     kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                           nn.BatchNorm2d(x), 
                           nn.ReLU()]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
        return nn.Sequential(*layers)
    
    def create_fc_layers(self):
        layers = nn.Sequential(nn.Linear(512*7*7, 4096),
                              nn.ReLU(),
                              nn.Dropout(p=0.5),
                              nn.Linear(4096, 4096),
                              nn.ReLU(),
                              nn.Dropout(p=0.5),
                              nn.Linear(4096, self.num_classes)
                              )
        return layers

In [ ]:
x = torch.rand(1, 3, 224, 224)
model = VGG16()
print(model(x).shape)

torch.Size([1, 1000])


In [ ]:
#Inception Net Architecture

class InceptionNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(InceptionNet, self).__init__()

        self.conv1 = conv_block(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.conv2 = conv_block(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1)
        self.max_pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception3a = Inception_block(in_channels=192, out_1x1=64, red_3x3=96, 
                                           out_3x3=128, red_5x5=16, out_5x5=32, out_1x1_pool=32)
        
        self.inception3b = Inception_block(in_channels=256, out_1x1=128, red_3x3=128, 
                                           out_3x3=192, red_5x5=32, out_5x5=96, out_1x1_pool=64)
        
        self.max_pool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception4a = Inception_block(in_channels=480, out_1x1=192, red_3x3=96, 
                                           out_3x3=208, red_5x5=16, out_5x5=48, out_1x1_pool=64)
        
        self.inception4b = Inception_block(in_channels=512, out_1x1=160, red_3x3=112, 
                                           out_3x3=224, red_5x5=24, out_5x5=64, out_1x1_pool=64)
        
        self.inception4c = Inception_block(in_channels=512, out_1x1=128, red_3x3=128, 
                                           out_3x3=256, red_5x5=24, out_5x5=64, out_1x1_pool=64)
        
        self.inception4d = Inception_block(in_channels=512, out_1x1=112, red_3x3=144, 
                                           out_3x3=288, red_5x5=32, out_5x5=64, out_1x1_pool=64)
        
        self.inception4e = Inception_block(in_channels=528, out_1x1=256, red_3x3=160, 
                                           out_3x3=320, red_5x5=32, out_5x5=128, out_1x1_pool=128)
        
        self.max_pool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception5a = Inception_block(in_channels=832, out_1x1=256, red_3x3=160, 
                                           out_3x3=320, red_5x5=32, out_5x5=128, out_1x1_pool=128)
        
        self.inception5b = Inception_block(in_channels=832, out_1x1=384, red_3x3=192, 
                                           out_3x3=384, red_5x5=48, out_5x5=128, out_1x1_pool=128)
        
        self.avg_pool = nn.AvgPool2d(kernel_size=7, stride=1)

        self.dropout = nn.Dropout(p=0.4)
        self.out = nn.Linear(1024, 1000)

    def forward(self, x):
        x = self.conv1(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.max_pool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.max_pool3(x)

        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.max_pool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avg_pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.out(x)
        return x

class Inception_block(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1_pool):
        super(Inception_block, self).__init__()
        self.branch1 = conv_block(in_channels, out_1x1, kernel_size=1)

        self.branch2 = nn.Sequential(conv_block(in_channels, red_3x3, kernel_size=1),
                                     conv_block(red_3x3, out_3x3, kernel_size=3, padding=1))
        
        self.branch3 = nn.Sequential(conv_block(in_channels, red_5x5, kernel_size=1),
                                     conv_block(red_5x5, out_5x5, kernel_size=5, padding=2))
        
        self.branch4 = nn.Sequential(nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
                                     conv_block(in_channels, out_1x1_pool, kernel_size=1))
        
    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)


class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()
        self.relu = nn.ReLU()
        self.conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, **kwargs)
        self.batch_norm = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        return self.batch_norm(self.relu(self.conv_layer(x)))

In [ ]:
x = torch.rand(1, 3, 224, 224)
model = InceptionNet()
print(model(x).shape)

torch.Size([1, 1000])


In [ ]:
# ResNet Architecture
class block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = 4

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.conv3 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.relu(self.batch_norm2(self.conv2(x)))
        x = self.relu(self.batch_norm3(self.conv3(x)))

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

class ResNet(nn.Module): #layers [3, 4, 6, 6] for ResNet50
    def __init__(self, block, layers, image_channels=3, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layers(block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layers(block, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layers(block, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layers(block, layers[3], out_channels=512, stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*4, num_classes)

    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avg_pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

    def _make_layers(self, block, num_residuals, out_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != out_channels*4:
            identity_downsample = nn.Sequential(nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels*4, 
                                                          kernel_size=1, stride=stride),
                                                nn.BatchNorm2d(out_channels*4),
                                                nn.ReLU())
        layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
        self.in_channels = out_channels*4

        for i in range(num_residuals - 1):
            layers.append(block(self.in_channels, out_channels))
        
        return nn.Sequential(*layers)

def ResNet50(image_channels=3, num_classes=1000):
    return ResNet(block, layers=[3, 4, 6, 3], image_channels=3, num_classes=1000)

In [ ]:
x = torch.rand(1, 3, 224, 224)
model = ResNet50()
print(model(x).shape)

torch.Size([1, 1000])
